# Общая информация

Данный блокнот используется дл преобразования видео в текст. Для этого используется модель [VideoLLaMA3-7B](https://huggingface.co/DAMO-NLP-SG/VideoLLaMA3-7B). Данная модель является мультимодальной, но охватывает только 2 модальности: видео и текст (в том числе, решает задачу OCR). Довольно большая смысловая часть сосредоточена в аудиопотоке, для его обработки использовалась модель [whisper-large-v3-turbo](https://huggingface.co/openai/whisper-large-v3-turbo).

## Информация о видекарте

In [15]:
!nvidia-smi

Fri May 23 10:58:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             54W /  400W |   18957MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [14]:
!pip install -q gpustat
!gpustat

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gpustat]
bc41b4e12ddf              Fri May 23 10:58:05 2025  550.54.15
[0] NVIDIA A100-SXM4-40GB | 34°C,   0 % | 18957 / 40960 MB |


## Подсчёты ресурсов при обработке

Как видно из описания выше, для обработки использовалась видеокарта `NVIDIA A100-SXM4-40GB`. Скорость обработки видео с помощью ансамбля выбранных моделей (`VideoLLaMA3-7B + whisper-large-v3-turbo`) на данном GPU составляет около 100 видео в час. Примерный расход ресурсных блоков для 1 сеанса примерно равен `6.86` в час.

## Необходимые предустановки

*этот `flash` подходит конкретно для А100

In [1]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.5 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linu

In [2]:
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.14.0 which is incompatible.


In [3]:
!pip install decord ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 116.8 MB/s eta 0:00:00


In [4]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 102.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803404 sha256=8ba909ec00fda553b28fe2317b8bc7741e85edc6c557e120e28c9cc55cfada41
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0


In [5]:
!pip install --upgrade pip
!pip install transformers datasets[audio] accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


## Основной скрипт

Настройки

In [29]:
import os
import gc
import time
import requests
import sqlite3
from tqdm import tqdm
import subprocess
import whisper
import warnings

VIDEO_FOLDER = "./videos"
os.makedirs(VIDEO_FOLDER, exist_ok=True)

DB_PATH = "video_descriptions.db"  # файл с бд
CSV_URL = "https://s3.ritm.media/hackaton-itmo/task.csv"
VIDEO_MODEL_NAME = "DAMO-NLP-SG/VideoLLaMA3-7B"  # модель для обработки видео
AUDIO_MODEL_NAME = "openai/whisper-large-v3"  # модель для обработки аудио

MAX_ATTEMPTS = 5

Здесь:
- `MAX_ATTEMPTS = 5` -- это количество раз которое модель пройдётся по одному и тому же файлу пока не выдаст корректное текстовое описание

Инициализация модели `VideoLLaMA3-7B`

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoModel, AutoImageProcessor

processor = AutoProcessor.from_pretrained(VIDEO_MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    VIDEO_MODEL_NAME,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

processing_videollama3.py:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

image_processing_videollama3.py:   0%|          | 0.00/21.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Videollama3Qwen2ForCausalLM(
  (model): Videollama3Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e

Инициализация модели `whisper-large-v3`

In [8]:
from transformers import AutoProcessor as WhisperProcessor, AutoModelForSpeechSeq2Seq, pipeline

# Загрузка Whisper Turbo
whisper_turbo_processor = WhisperProcessor.from_pretrained(AUDIO_MODEL_NAME)
whisper_turbo_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    AUDIO_MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_safetensors=True
).to("cuda")

whisper_pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper_turbo_model,
    tokenizer=whisper_turbo_processor.tokenizer,
    feature_extractor=whisper_turbo_processor.feature_extractor,
    return_timestamps=False,
    max_new_tokens=128,
    torch_dtype=torch.float16,
    device=0
)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(


Получаем список ссылок на видео

In [9]:
video_list_url = "https://s3.ritm.media/hackaton-itmo/task.csv"
video_urls = requests.get(video_list_url).text.strip().splitlines()[1:]
video_urls = [u.strip() for u in video_urls if u.startswith("http")]

Промпт

In [10]:
base_question = (
    "Опиши подробно по-русски, что происходит на видео от начала и до конца, включая действия, объекты, людей, эмоции и окружение. "
    "Обрати внимание, что в видео могут использоваться сленговые выражения, субтитры, названия или текстовые вставки. "
    "Также в кадре могут появляться персонажи или образы из фильмов, игр и других медиа — обязательно укажи, если они есть. "
    "Твоя цель — создать описание, из которого легко выделить ключевые темы, события и теги. Используй не менее 150 слов."
)

Функция генерации описания

In [11]:
def get_valid_response(conversation):
    for attempt in range(MAX_ATTEMPTS):
        try:
            inputs = processor(conversation=conversation, return_tensors="pt")
            inputs = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
            if "pixel_values" in inputs:
                inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)

            with torch.no_grad():
                output_ids = model.generate(
                    **inputs,
                    max_new_tokens=512,
                    do_sample=True,         # включить семплирование
                    temperature=0.7,
                    top_p=0.9,              # nucleus sampling: учитываются токены, набирающие 90% вероятности
                    top_k=50,               # ограничение на число возможных токенов
                    repetition_penalty=1.2, # штраф за повторы
                    length_penalty=1.0,     # не поощряет/не штрафует длинные ответы
                    eos_token_id=processor.tokenizer.eos_token_id  # если явно нужен
                )

            response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
            response_lower = response.lower()

            # Проверка на отказ или слишком короткий ответ
            if not response or len(response.split()) < 20:
                print(f"Слишком короткий ответ (попытка {attempt+1})")
                time.sleep(1)
                continue

            if any(phrase in response_lower for phrase in [
                "не могу", "не способен", "недопустимо", "не поддерживается"
            ]):
                print(f"Модель отказалась отвечать (попытка {attempt+1})")
                new_conversation = conversation.copy()
                new_conversation[-1]["content"][1]["text"] = (
                    "Опиши подробно по-русски, что происходит на видео от начала и до конца, включая действия, объекты, людей, эмоции и окружение. "
                    "Твоя цель — создать описание, из которого легко выделить ключевые темы и теги. Используй не менее 150 слов. "
                    "Опиши действия, объекты, людей и окружение. Не упоминай запреты или ограничения."
                )

                conversation = new_conversation
                time.sleep(1)
                continue

            return response

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print("OOM — память закончилась.")
                return "[FAILED] CUDA out of memory"
            else:
                raise e

    return "[FAILED] Не удалось получить описание после 3 попыток"

Инициализация SQLite

In [26]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS videos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        filename TEXT UNIQUE,
        description TEXT,
        transcription TEXT
    )
""")

# Добавляем столбец, если запуск повторный
try:
    cursor.execute("ALTER TABLE videos ADD COLUMN transcription TEXT")
except sqlite3.OperationalError:
    pass  # уже есть

conn.commit()

Обрабатываемый фрагмент внутри таблицы

In [33]:
OFFSET = 0  # индекс, с которого начинать обрабатывать
LIMIT = 5    # сколько нужно обработать

Функция обработки видео

In [31]:
def process_batch(OFFSET: int, LIMIT: int):
    import pandas as pd
    from io import StringIO

    # Загрузка таблицы
    response = requests.get(CSV_URL)
    df = pd.read_csv(StringIO(response.text))
    video_urls = df['video_link'].tolist()

    # Соединение с БД
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    progress_bar = tqdm(video_urls[OFFSET:OFFSET + LIMIT], desc="Прогресс", position=0)

    for video_url in progress_bar:
        filename = os.path.basename(video_url)
        video_path = os.path.join(VIDEO_FOLDER, filename)

        # Пропуск, если уже есть в БД
        cursor.execute("SELECT 1 FROM videos WHERE filename = ?", (filename,))
        if cursor.fetchone():
            progress_bar.set_description(f"Уже есть: {filename}")
            continue

        # Скачивание видео
        try:
            r = requests.get(video_url, stream=True)
            with open(video_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        except Exception as e:
            progress_bar.set_description(f"Ошибка скачивания: {filename}")
            continue

        progress_bar.set_description(f"▶ Обрабатываем: {filename}")

        try:
            # Промпт к модели
            conversation = [
                {"role": "system", "content": base_question},
                {
                    "role": "user",
                    "content": [
                        {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 64}},
                        {"type": "text", "text": base_question}
                    ]
                }
            ]

            # Получение описания
            response_text = get_valid_response(conversation)

            # Аудио
            audio_path = video_path.replace(".mp4", ".wav")
            try:
                subprocess.run([
                    "ffmpeg", "-y", "-i", video_path,
                    "-vn", "-acodec", "pcm_s16le",
                    "-ar", "16000", "-ac", "1",
                    audio_path
                ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

                transcription_result = whisper_pipe(
                    audio_path,
                    return_timestamps=True,
                    generate_kwargs={"language": "ru"}
                )
                transcription = transcription_result.get("text", "").strip()

            except Exception as e:
                transcription = "[FAILED]"
                progress_bar.set_description(f"Ошибка аудио: {filename}")

            # Сохраняем в БД
            cursor.execute(
                "INSERT OR IGNORE INTO videos (filename, description, transcription) VALUES (?, ?, ?)",
                (filename, response_text, transcription)
            )
            cursor.execute(
                "UPDATE videos SET description = ?, transcription = ? WHERE filename = ?",
                (response_text, transcription, filename)
            )
            conn.commit()

            print(f"\n📹 Описание ({filename}):\n{response_text}\n")
            print(f"🔊 Транскрипция ({filename}):\n{transcription}\n")

            try:
                os.remove(audio_path)
            except:
                pass

        except Exception as e:
            progress_bar.set_description(f"Ошибка при обработке: {filename}")

        finally:
            try:
                os.remove(video_path)
            except:
                pass
            torch.cuda.empty_cache()
            gc.collect()

    conn.close()
    print(f"Обработка диапазона {OFFSET}–{OFFSET + LIMIT} завершена.")

Обработка видео

In [35]:
process_batch(OFFSET, LIMIT)

▶ Обрабатываем: 000a19ce-98ac-4054-b600-23362f8edfdc.mp4:   0%|          | 0/5 [00:03<?, ?it/s]

Слишком короткий ответ (попытка 1)
Слишком короткий ответ (попытка 2)

📹 Описание (000a19ce-98ac-4054-b600-23362f8edfdc.mp4):
В видео рассказывают историю о двух людях, которые прилетели в Монако и их встрече с президентом страны. В начале видео первый человек рассказывает, как они приехали на день рождения дочери президента и были встретлены коротким человеком, который был очень приветливым и классным. Затем он показывает картинку с лифтом из отеля к морю, слепой стенкой для празднования дня города, апартаментами и бассейном перед столиком.

Далее, второй человек говорит, что они должны быть там, потому что у них было два выступления в Высшей Лиге, но не команды КВН «Казахи». Когда это становится известно, первый человек удивляется и делает вывод, что они — команда КВН «Камышков».

Также в видео есть образы из фильмов, игр или других медиа: зверь, который находится рядом со вторым человеком; телевизор на фоне первой части истории; стены и окна с ярко-синими стеклами.

Используемые тек

▶ Обрабатываем: 000b132f-9c3a-4b57-840b-5dad1ef2de08.mp4:  20%|██        | 1/5 [01:02<03:59, 59.89s/it]

Слишком короткий ответ (попытка 1)

📹 Описание (000b132f-9c3a-4b57-840b-5dad1ef2de08.mp4):
В видео группа людей занимается танцем в роскошном зале. В начале они сначала выполняют упражнения на пол, затем стоя, и после этого переходят к выполнению мотивов. Обычно все делают движения одновременно, но иногда разбиваются по группам. Часто используя белый цвет в оформлении зала, можно видеть много ленточных огней на одной из стен, а также большую колонну. Также есть баррикадные столбы в середине помещения.

🔊 Транскрипция (000b132f-9c3a-4b57-840b-5dad1ef2de08.mp4):
Моя душа Песня «Солнце»



▶ Обрабатываем: 001423e2-43b1-450d-9c4d-068e230cd404.mp4:  40%|████      | 2/5 [01:23<01:51, 37.27s/it]

...


📹 Описание (0014872e-6efb-4378-87f8-820acfb6801b.mp4):
В видео можно увидеть чат-сообщение между двумя людьми. На экране показано сообщение от пользователя "Артем Ураганов" с текстом "Мой южный 🌈". Затем появляется ответ на это сообщение, которое было отправлено в 07:13. В последующих кадрах можно видеть сообщения и реакции пользователей о том, что один из них приветствует другого как "мой душа", а также выражает любовь.

Общие темы видео связаны с общением, знакомствами и выражением чувств.

🔊 Транскрипция (0014872e-6efb-4378-87f8-820acfb6801b.mp4):
Продолжение следует...



▶ Обрабатываем: 002a835c-0bed-45ab-977b-1d393b0814a7.mp4:  80%|████████  | 4/5 [01:56<00:23, 23.02s/it]


📹 Описание (002a835c-0bed-45ab-977b-1d393b0814a7.mp4):
В начале видео представлен анимационный персонаж с двумя оттенками волос (одна сторона коричневая, другая - фиолетовая), уши в виде кошачьих и зелёные глаза. Персонаж находится на простом фоне, где она начинает петь и танцевать, излучая радость и энергии. В текстовом представлении "Александра" указывается, что это её имя.

Затем персонаж превращается в другую фигуру с более скромным костюмом для игры на гитаре, как именовано через появившиеся слова "Хорош" или "Хорем". Она продолжает музыкальное исполнение, при этом сохраняя эмоциональную открытость и радость.

Таким образом, ключевые темы данного видео заключаются в музыке, самовыражении и веселье, подчеркиваемые яркими манипуляциями персонажей.

🔊 Транскрипция (002a835c-0bed-45ab-977b-1d393b0814a7.mp4):
Я не пойду в школу, делаю уроки, я не пойду пить колу, это бедно для здоровья. Я хочу в питьли, хотя сейчас 20-го я школа, но все это вон снисти на старую.



▶ Обрабатываем: 002a835c-0bed-45ab-977b-1d393b0814a7.mp4: 100%|██████████| 5/5 [02:12<00:00, 26.53s/it]

Обработка диапазона 0–5 завершена.


## Загрузка requirements

In [18]:
!pip freeze > requirements.txt

In [39]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Вывод пропущенных в ходе обработки видео

In [ ]:
import pandas as pd
import json
import requests
from io import StringIO

# 1. Загрузка CSV
url = "https://s3.ritm.media/hackaton-itmo/task.csv"
response = requests.get(url)
df_csv = pd.read_csv(StringIO(response.text))

# Отрезаем нужный диапазон
df_slice = df_csv.iloc[OFFSET:OFFSET + LIMIT].copy()

# 2. Загрузка JSON с описаниями
with open(json_file, "r", encoding="utf-8") as f:
    descriptions = json.load(f)

described_filenames = {item["filename"] for item in descriptions}

# 3. Проверка пропущенных
missing_dict = {}
for relative_idx, row in df_slice.iterrows():
    filename = row['video_link'].strip().split('/')[-1]
    if filename not in described_filenames:
        missing_dict[filename] = relative_idx  # абсолютный индекс в CSV

# 4. Вывод
print(f"Всего пропущено: {len(missing_dict)} файлов\n")
print(json.dumps(missing_dict, indent=2, ensure_ascii=False))

Всего пропущено: 0 файлов

{}


## Скачать JSON

Загрузить бд в JSON

In [22]:
import sqlite3
import pandas as pd

# Открываем соединение
conn = sqlite3.connect("video_descriptions.db")

# Загружаем данные в DataFrame
df = pd.read_sql_query("SELECT * FROM videos", conn)

# Сохраняем в JSON
json_file = f"video_descriptions_{OFFSET}_{OFFSET+LIMIT}.json"
df.to_json(json_file, orient="records", force_ascii=False, indent=2)

conn.close()

Скачать JSON

In [ ]:
from google.colab import files
files.download(json_file)

## Повторный запуск для группы видео

In [38]:
JSON_OUTPUT = f"video_descriptions_{OFFSET}_{OFFSET + LIMIT}.json"

# --- Загрузка таблицы CSV ---
response = requests.get(CSV_URL)
df = pd.read_csv(StringIO(response.text))

# --- Загружаем существующие результаты (если есть) ---
if os.path.exists(JSON_OUTPUT):
    with open(JSON_OUTPUT, "r", encoding="utf-8") as f:
        results = json.load(f)
    already_done = {item["filename"] for item in results}
else:
    results = []
    already_done = set()

# --- Формируем список пропущенных в диапазоне OFFSET:OFFSET+LIMIT ---
missing_dict = {}
for idx in range(OFFSET, OFFSET + LIMIT):
    video_url = df.loc[idx, 'video_link']
    filename = video_url.strip().split('/')[-1]
    if filename not in already_done:
        missing_dict[filename] = idx

# --- Обработка только пропущенных ---
for filename, idx in tqdm(missing_dict.items(), desc="Обработка"):
    video_url = df.loc[int(idx), 'video_link'].strip()
    video_path = os.path.join(VIDEO_FOLDER, filename)

    try:
        # Скачивание видео
        r = requests.get(video_url, stream=True)
        with open(video_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

        # Промпт
        conversation = [
            {"role": "system", "content": base_question},
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 64}},
                    {"type": "text", "text": base_question}
                ]
            }
        ]

        # Получение описания
        response = get_valid_response(conversation)

        # Аудио
        audio_path = video_path.replace(".mp4", ".wav")
        try:
            subprocess.run([
                "ffmpeg", "-y", "-i", video_path,
                "-vn", "-acodec", "pcm_s16le",
                "-ar", "16000", "-ac", "1",
                audio_path
            ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

            transcription_result = whisper_pipe(
                audio_path,
                return_timestamps=True,
                generate_kwargs={"language": "ru"}
            )
            transcription = transcription_result.get("text", "").strip()

        except Exception as e:
            transcription = "[FAILED]"
            print(f"[{filename}] Ошибка аудио: {e}")

        # Сохраняем результат
        results.append({
            "id": int(idx) + 1,
            "filename": filename,
            "description": response,
            "transcription": transcription
        })

    except Exception as e:
        print(f"[{filename}] Пропущено: {e}")
        continue
    finally:
        try:
            os.remove(video_path)
            os.remove(audio_path)
        except:
            pass
        torch.cuda.empty_cache()
        gc.collect()

    # Промежуточное сохранение
    with open(JSON_OUTPUT, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

# --- Финал ---
with open(JSON_OUTPUT, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Завершена обработка диапазона {OFFSET}-{OFFSET + LIMIT}.")

Обработка: 0it [00:00, ?it/s]

Завершена обработка диапазона 0-5.


## Восстановить бд из json

In [ ]:
import sqlite3
import json

DB_PATH = "video_descriptions.db"
JSON_PATH = "video_descriptions_900.json"  # путь к JSON

# Загружаем JSON
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# Создаём соединение и пересоздаём таблицу
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS videos")
cursor.execute("""
    CREATE TABLE videos (
        id INTEGER PRIMARY KEY,
        filename TEXT UNIQUE,
        description TEXT
    )
""")

# Добавляем записи
for row in data:
    cursor.execute(
        "INSERT INTO videos (id, filename, description) VALUES (?, ?, ?)",
        (row["id"], row["filename"], row["description"])
    )

conn.commit()
conn.close()

print(f"Загружено {len(data)} записей в {DB_PATH}")

Загружено 994 записей в video_descriptions.db


## Удалить базу данных

In [25]:
db_path = "video_descriptions.db"

if os.path.exists(db_path):
    os.remove(db_path)
    print(f"База данных '{db_path}' удалена.")
else:
    print(f"Файл базы данных '{db_path}' не найден.")

База данных 'video_descriptions.db' удалена.
